In [4]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import webbrowser

In [30]:
import os

In [28]:
from selenium import webdriver

In [69]:
from seleniumwire import webdriver

In [119]:
chrome_driver_path = os.path.abspath("./chromedriver.exe")
prefs = {'profile.default_content_setting_values': {'cookies': 2, 'images': 2, 'plugins': 2, 'popups': 2,
                                                    'geolocation': 2, 'notifications': 2,
                                                    'auto_select_certificate': 2, 'fullscreen': 2,
                                                    'mouselock': 2, 'mixed_script': 2, 'media_stream': 2,
                                                    'media_stream_mic': 2, 'media_stream_camera': 2,
                                                    'protocol_handlers': 2, 'ppapi_broker': 2,
                                                    'automatic_downloads': 2, 'midi_sysex': 2,
                                                    'push_messaging': 2, 'ssl_cert_decisions': 2,
                                                    'metro_switch_to_desktop': 2,
                                                    'protected_media_identifier': 2, 'app_banner': 2,
                                                    'site_engagement': 2, 'durable_storage': 2,
                              
                                                    'javascript': 2}}

options = webdriver.ChromeOptions()
options.add_experimental_option('prefs', prefs)
options.add_argument("start-maximized")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument('headless')
options.add_argument("disable-gpu")
# options.add_argument(
#     "user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90  Safari/537.36")

driver = webdriver.Chrome(executable_path=chrome_driver_path,
                          chrome_options=options)

In [142]:
driver_wire = webdriver.Chrome(executable_path=chrome_driver_path,
                          chrome_options=options)

In [115]:
# config definition
auth_key = "e80e0f9ecbc28594bf7acb6663da89e903e9865e"
company_code="024110"
start_date="1990101"

def get_report_list(auth_key, company_code, start_date):
    url = "http://dart.fss.or.kr/api/search.xml?auth="+auth_key+"&crp_cd="+company_code+"&start_dt="+start_date+"&bsn_tp=A001&bsn_tp=A002&bsn_tp=A003"

    response = requests.get(url)

    result = response.text

    xmlsoup=BeautifulSoup(result,'html.parser')

    data = pd.DataFrame()

    te = xmlsoup.findAll("list")

    for t in te:
        temp=pd.DataFrame(([[t.crp_cls.string,t.crp_nm.string,t.crp_cd.string,t.rpt_nm.string,
            t.rcp_no.string,t.flr_nm.string,t.rcp_dt.string, t.rmk.string]]),
            columns=["crp_cls","crp_nm","crp_cd","rpt_nm","rcp_no","flr_nm","rcp_dt","rmk"])
        data=pd.concat([data,temp])

    data.reset_index(drop=True, inplace=True)
    
    return data

In [252]:
def get_link_fs_annotation(report_url):
    driver.get(report_url)

    html = driver.page_source
    data = BeautifulSoup(html, 'html.parser')

    cnt = 1
    for i in data.find_all('li', {'class':"x-tree-node"}):
        if i.get_text() == '3. 연결재무제표 주석':
            print(cnt)
            break
        cnt += 1

    driver_wire.get(report_url)
    driver_wire.find_element_by_xpath("//*[@id='ext-gen10']/div/li[5]/ul/li[3]/div").click()
    response_list = list()

    for request in driver_wire.requests:
        if request.response:
                response_list.append(request.path)

    temp1 = response_list[-1].split('&')
    temp = response_list[-1].split('&')[2].split('=')

    temp[-1] = str(cnt)
    temp1[2] = '='.join(temp)

    link_url = '&'.join(temp1)

    driver.get(link_url)
    html = driver.page_source

    data = BeautifulSoup(html, 'html.parser')
    
    return data

In [116]:
df = get_report_list(auth_key, company_code, start_date)

In [254]:
url_user="http://dart.fss.or.kr/dsaf001/main.do?rcpNo="+df['rcp_no'][5]

In [255]:
url_user

'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20190401004404'

In [253]:
get_link_fs_annotation(url_user)

14


<html style="border:0"><head>
<title></title>
<meta content="IE=EmulateIE8" http-equiv="X-UA-Compatible"/>
<meta content="gdi" http-equiv="X-UA-TextLayoutMetrics"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<link href="/css/report_xml.css" rel="stylesheet" type="text/css"/>
</head>
<body bgcolor="#FFFFFF">
<p><br/></p>
<p class="section-2"><a name="toc1">3. 연결재무제표 주석</a></p>
<p><br/></p>
<table class="nb" width="600">
<colgroup>
<col width="600"/>
</colgroup>
<tbody>
<tr>
<td align="CENTER" height="20" valign="TOP" width="600">제59기 1분기 : 2019년 <span style="color:#FFFFFF;">1</span>3월 31일 현재
</td>
</tr>
<tr>
<td align="CENTER" height="20" valign="TOP" width="600">제58기 <span style="color:#FFFFFF;">1분기</span> : 2018년 12월 31일 현재
</td>
</tr>
<tr>
<td height="20" valign="TOP" width="600">중소기업은행과 그 종속기업</td>
</tr>
</tbody>
</table>
<p><br/></p>
<p>1. 일반적 사항</p>
<p><span><br/>중소기업은행(이하 "지배기업")은 1961년 중소기업은행법에 의거 중소기업자에 대한효율적인 신용제도를 확립함으로써 중소기업자의 자주적인 경제활동을 원활히 하고 그 경제

In [242]:
data

<html style="border:0"><head>
<title></title>
<meta content="IE=EmulateIE8" http-equiv="X-UA-Compatible"/>
<meta content="gdi" http-equiv="X-UA-TextLayoutMetrics"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<link href="/css/report_xml.css" rel="stylesheet" type="text/css"/>
</head>
<body bgcolor="#FFFFFF">
<p><br/></p>
<p class="section-2"><a name="toc1">3. 연결재무제표 주석</a></p>
<p><br/></p>
<table class="nb" width="600">
<colgroup>
<col width="600"/>
</colgroup>
<tbody>
<tr>
<td align="CENTER" height="20" valign="TOP" width="600">제59기 1분기 : 2019년 <span style="color:#FFFFFF;">1</span>3월 31일 현재
</td>
</tr>
<tr>
<td align="CENTER" height="20" valign="TOP" width="600">제58기 <span style="color:#FFFFFF;">1분기</span> : 2018년 12월 31일 현재
</td>
</tr>
<tr>
<td height="20" valign="TOP" width="600">중소기업은행과 그 종속기업</td>
</tr>
</tbody>
</table>
<p><br/></p>
<p>1. 일반적 사항</p>
<p><span><br/>중소기업은행(이하 "지배기업")은 1961년 중소기업은행법에 의거 중소기업자에 대한효율적인 신용제도를 확립함으로써 중소기업자의 자주적인 경제활동을 원활히 하고 그 경제

In [ ]:
/report/viewer.do
?rcpNo=20191114002595
&dcmNo=6961369
&eleId=14
&offset=707322
&length=1272135
&dtd=dart3.xsd

In [ ]:
/report/viewer.do
?rcpNo=20190515002645
&dcmNo=6741440
&eleId=14
&offset=695615
&length=1198714
&dtd=dart3.xsd

eleId